# Math2 dataset preprocessing
### Import

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import pandas as pd
import numpy as np
from liriscat.dataset.preprocessing_utilities import *
from liriscat.utils import *
utils.set_seed(0)

### Load and merge dataset

In [ ]:
raw_data = pd.read_csv('../1-raw_data/math_2.csv', encoding="ISO-8859-15", low_memory=False)

raw_data = raw_data.drop(columns=['Unnamed: 0.2', 'Unnamed: 0',
       'Unnamed: 0.1', 'start_time', 'Unnamed: 0.1.1', 'qr',
       'qr_compl'])

raw_data =  raw_data.rename(columns={"skill_id": "dimension_id"})
raw_data.head()

### Split rows with several dimensions

In [ ]:
raw_data['dimension_id'] = raw_data['dimension_id'].apply(lambda x: set(map(int, x.strip('{}').split(', '))))
df_expanded = raw_data.explode('dimension_id').reset_index(drop=True)
df_expanded.head()

### Early stats

In [ ]:
print(len(df_expanded))
all_data = df_expanded.dropna(subset=['user_id','item_id','correct','dimension_id'])
print(len(all_data))

In [ ]:
stat_unique(all_data, None)
stat_unique(all_data, ['user_id', 'item_id'])
stat_unique(all_data, 'user_id')
stat_unique(all_data, 'item_id')
stat_unique(all_data, 'dimension_id')

### Clean Nan values

In [ ]:
cleaned_data = all_data.dropna(subset=['user_id','item_id','correct','dimension_id'], axis='index')

### Remove duplicated user and id rows

In [ ]:
unduplicated_data = remove_duplicates(cleaned_data,key_attrs=['user_id','item_id','dimension_id'],agg_attrs=[])

### Filter Data

In [ ]:
filtered_data_0 = unduplicated_data
filtered_q_nb = 1
filtered_u_nb = 0
filtered_d_nb = 0
while filtered_q_nb!=0 or filtered_u_nb!=0 or filtered_d_nb!=0 : 
    # filter items
    filtered_data_1, filtered_q_nb = densify(filtered_data_0,'item_id','user_id',5)
    # filter users
    min_nb_users_logs = 20
    filtered_data_2, filtered_u_nb = densify(filtered_data_1,'user_id','item_id',min_nb_users_logs)
    # filter knowledges
    filtered_data_3, filtered_d_nb = densify(filtered_data_2,'dimension_id','item_id',1)
    filtered_data_0 = filtered_data_3
    
stat_unique(filtered_data_0, None)
stat_unique(filtered_data_0, ['user_id', 'item_id'])
stat_unique(filtered_data_0, 'user_id')
stat_unique(filtered_data_0, 'item_id')
stat_unique(filtered_data_0, 'dimension_id')

### Encoding

In [ ]:
# renumber the users
u_enc_data, u2n = encode_attr(filtered_data_0, "user_id")

In [ ]:
# renumber the items
q_enc_data, q2n = encode_attr(u_enc_data, "item_id")

In [ ]:
# renumber the dimensions
d_enc_data, d2n = encode_attr(q_enc_data, "dimension_id")

### Maps creation

In [ ]:
q2k, k2q = create_q2k(d_enc_data)

### Rescaling responses

In [ ]:
resc_data = d_enc_data
resc_data['correct'] = resc_data['correct'].astype(float)
min_val = resc_data['correct'].min()
max_val = resc_data['correct'].max()
resc_data['correct'] = (resc_data['correct']-min_val)/(max_val-min_val) +1
print(f'min value : {resc_data["correct"].min()}, max value : {resc_data["correct"].max()}')

### Get final stats and create metadata

In [ ]:
stat_unique(resc_data, None)
stat_unique(resc_data, ['user_id', 'item_id'])
stat_unique(resc_data, 'user_id')
stat_unique(resc_data, 'item_id')
stat_unique(resc_data, 'dimension_id')
stat_unique(resc_data, 'correct')

resc_data_dim_grouped_items = resc_data.groupby("dimension_id")["item_id"].nunique()
resc_data_dim_grouped_users = resc_data.groupby("dimension_id")["user_id"].nunique()
print('#questions/category: {} & {} &  {}'.format(np.min(resc_data_dim_grouped_items),int(np.round(np.mean(resc_data_dim_grouped_items))),np.max(resc_data_dim_grouped_items)))
print('#users/category: {} & {} &  {}'.format(np.min(resc_data_dim_grouped_users),int(np.round(np.mean(resc_data_dim_grouped_users))),np.max(resc_data_dim_grouped_users)))

resc_data_dim_grouped_items = resc_data.groupby("item_id")["dimension_id"].nunique()
resc_data_dim_grouped_users = resc_data.groupby("item_id")["user_id"].nunique()
print('#categorys/question: {} & {} &  {}'.format(np.min(resc_data_dim_grouped_items),int(np.round(np.mean(resc_data_dim_grouped_items))),np.max(resc_data_dim_grouped_items)))
print('#users/question: {} & {} &  {}'.format(np.min(resc_data_dim_grouped_users),int(np.round(np.mean(resc_data_dim_grouped_users))),np.max(resc_data_dim_grouped_users)))

resc_data_dim_grouped_items = resc_data.groupby("user_id")["item_id"].nunique()
resc_data_dim_grouped_users = resc_data.groupby("user_id")["dimension_id"].nunique()
print('#questions/users: {} & {} &  {}'.format(np.min(resc_data_dim_grouped_items),int(np.round(np.mean(resc_data_dim_grouped_items))),np.max(resc_data_dim_grouped_items)))
print('#categorys/users: {} & {} &  {}'.format(np.min(resc_data_dim_grouped_users),int(np.round(np.mean(resc_data_dim_grouped_users))),np.max(resc_data_dim_grouped_users)))

In [ ]:
import matplotlib.pyplot as plt

# draw a histogram of the age column
(resc_data['correct']-1).hist(bins=20)

# add labels and title
plt.xlabel('Responses')
plt.ylabel('Number')
plt.title('Distribution of Responses')
plt.savefig('../4-figs/distribution_math2.png')


In [ ]:
resc_data_dim_grouped_items = resc_data.groupby("item_id")["dimension_id"].nunique()
resc_data_dim_grouped_users = resc_data.groupby("item_id")["user_id"].nunique()
metadata = get_metadata(resc_data,["user_id", "item_id", "dimension_id"])
metadata['min_nb_users_logs'] = min_nb_users_logs
metadata['max_nb_categories_per_question'] = int(np.max(resc_data_dim_grouped_items))

### Save data, concept map and metadata

In [ ]:
new_order = ['user_id','item_id', 'correct', 'dimension_id']
resc_data = resc_data[new_order]

In [ ]:
# save selected data
resc_data.to_csv('../1-raw_data/math2_selected_data.csv', index=False)

In [ ]:
# Convert int64 keys to str
q2k_str_keys = {str(key): value for key, value in q2k.items()}

# Convert sets to lists in the dictionary
q2k_json_serializable = {}
for key, value in q2k_str_keys.items():
    if isinstance(value, set):
        q2k_json_serializable[key] = list(value)
    else:
        q2k_json_serializable[key] = value

# Save concept map
with open('../2-preprocessed_data/math2_concept_map.json', 'w') as f:
    json.dump(q2k_json_serializable, f)

In [ ]:
# save metadata
with open('../2-preprocessed_data/math2_metadata.json', 'w') as f:
    json.dump(metadata, f)

## parse data

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from liriscat import utils
utils.setuplogger(verbose = True,log_path = "../../logs/", log_name="liriscat")

In [ ]:
import pandas as pd
from liriscat.dataset.preprocessing_utilities import *
resc_data = pd.read_csv('../1-raw_data/math2_selected_data.csv')

In [ ]:
train, valid, test = split_users(resc_data, 5)

## save data

In [ ]:
for i_fold, train_fold in enumerate(train):
    save_df_to_csv(train[i_fold], f'../2-preprocessed_data/math2_train_{i_fold}.csv')
    save_df_to_csv(valid[i_fold], f'../2-preprocessed_data/math2_valid_{i_fold}.csv')
    save_df_to_csv(test[i_fold], f'../2-preprocessed_data/math2_test_{i_fold}.csv')

## Train IMPACT model

In [ ]:
from liriscat.utils import generate_eval_config
import json
from IMPACT.dataset import LoaderDataset as IMPACT_dataset
from liriscat.CDM import *
from IMPACT import model
from liriscat.dataset.preprocessing_utilities import *


folds_nb = 2
dataset_name="math2"

# Set all the required parameters ---------------
IMPACT_config = generate_eval_config(num_epochs=200, save_params=True, dataset_name=dataset_name, embs_path="../../embs/"+dataset_name, params_path="../../ckpt/"+dataset_name,  learning_rate=0.01885, lambda_=2e-7, batch_size=2048,valid_metric='mi_acc', pred_metrics=["mi_acc"])

concept_map = json.load(open(f'../2-preprocessed_data/{IMPACT_config["dataset_name"]}_concept_map.json', 'r'))
concept_map = {int(k): [int(x) for x in v] for k, v in concept_map.items()}

metadata = json.load(open(f'../2-preprocessed_data/{IMPACT_config["dataset_name"]}_metadata.json', 'r'))


# Conversion helper that builds a DataFrame with specific dtypes and returns records.
def convert_to_records(data):
    df = pd.DataFrame(data, columns={'user_id': int, 'item_id': int, 'category_id': int,'correct': float, })
    return df.to_records(index=False, column_dtypes={'user_id': int, 'item_id': int, 'correct': float, 'category_id': int})

In [ ]:
for i in range(folds_nb):
    # Merge the training and validation data for the current fold.
    train_valid_df = pd.concat([train[i], valid[i]])
    # Split the merged data horizontally into train and validation sets.

    quadruplet = quadruplet_format(train_valid_df)
    train_data, valid_data, test_data = split_data_vertically(quadruplet, test_prop=0.2,valid_prop=0.1,folds_nb=5)

    # Convert each split in one step using the helper.
    horizontal_train = convert_to_records(train_data[0])
    horizontal_valid = convert_to_records(valid_data[0])
    horizontal_test = convert_to_records(test_data[0])

    impact_train_data = IMPACT_dataset(horizontal_train, concept_map, metadata)
    impact_valid_data = IMPACT_dataset(horizontal_test, concept_map, metadata)

    IMPACT_config['i_fold'] = i
    algo = model.IMPACT(**IMPACT_config)
    algo.init_model(impact_train_data, impact_valid_data)
    algo.train(impact_train_data, impact_valid_data)
    print(algo.evaluate_predictions(impact_valid_data))